In [66]:
from datapath_tool import *
from toolfunc import *

In [9]:
savedatapath = r'/home/pi/Documents/data/asoproject/forreg20190913'
if not os.path.exists(savedatapath):
    os.mkdir(savedatapath)

# 1 生成两个回归用数据表
* 第一个以td分类为准生成diff
* 第二个以id_category20190913分类为准生成diff

## 1.0 公用数据，需求数据

In [33]:
dc_cmd = 'select * from category_demand_td_20190820'


dc = pd.read_sql(dc_cmd,conn).drop(columns = 'datahash')
print('dc:\n',dc.shape,'\n',dc.columns,'\n',dc.head())




dc:
 (264, 16) 
 Index(['type_code', 'date', 'app_m', 'app_v', 'apptype_m', 'apptype_v',
       'gtype_m', 'gtype_v', 'game_m', 'game_v', 'rgame_m', 'rgame_v',
       'rgtype_m', 'rgtype_v', 'type_ratio_m', 'type_ratio_v'],
      dtype='object') 
   type_code      date      app_m       app_v  apptype_m    apptype_v  \
0   T200100  20170131  17.446539   72.917478  43.209772  1557.470566   
1   T200200  20170131  20.021810  125.293308  49.889057  2699.840168   
2   T200300  20170131  18.414923   76.163659  44.034374  1596.993089   
3   T200400  20170131  17.779710   86.642816  47.669203  2066.743396   
4   T200500  20170131  19.771155   78.147914  48.962644  2931.923231   

    gtype_m   gtype_v    game_m      game_v   rgame_m   rgame_v  rgtype_m  \
0  4.382704  5.394237  7.025517   44.885675  0.185725  0.014252  0.284812   
1  4.741427  8.144833  8.280005  108.861570  0.193531  0.019521  0.304918   
2  4.647341  5.291457  7.451149   48.776628  0.189529  0.012930  0.286992   
3  4.647585

In [20]:
dcgp_cmd = 'select * from game_demand_perf_td_20190820'
pkgid_cmd = 'select * from pkgname_appleid'
dcgp = pd.read_sql(dcgp_cmd,conn).drop(columns = 'datahash').drop(columns = ['appHash','id'])
pkgid = pd.read_sql(pkgid_cmd,conn)[['pkgName','type_code','appleid']]
print('dcgp\n', dcgp.shape,'\n',dcgp.columns)
print('pkgid\n',pkgid.shape)
dcgpid = dcgp.merge(pkgid,on = ['pkgName','type_code'], how = 'left').dropna(subset = ['appleid'])
print('dcgpid\n',dcgpid.shape,'\n',dcgpid.columns)

dcgp
 (85185, 21) 
 Index(['pkgName', 'type', 'type_code', 'date', 'user_num', 'active_num',
       'cover_num', 'mau', 'coverage', 'g_app_m', 'g_app_v', 'g_apptype_m',
       'g_apptype_v', 'g_gtype_m', 'g_gtype_v', 'g_game_m', 'g_game_v',
       'g_rgame_m', 'g_rgame_v', 'g_rgtype_m', 'g_rgtype_v'],
      dtype='object')
pkgid
 (8031, 3)
dcgpid
 (46338, 22) 
 Index(['pkgName', 'type', 'type_code', 'date', 'user_num', 'active_num',
       'cover_num', 'mau', 'coverage', 'g_app_m', 'g_app_v', 'g_apptype_m',
       'g_apptype_v', 'g_gtype_m', 'g_gtype_v', 'g_game_m', 'g_game_v',
       'g_rgame_m', 'g_rgame_v', 'g_rgtype_m', 'g_rgtype_v', 'appleid'],
      dtype='object')


In [26]:
perf = dcgpid[['appleid','date','active_num','cover_num']]
print('perf:\n',perf.head())
user_num = dcgp[['date','user_num']].drop_duplicates()
print('user_num:\n',user_num.shape,'\n',user_num.head())
perfgroup = perf.groupby(['appleid','date'])
perf1 = perfgroup.agg(sum).reset_index()
print('perf1:\n',perf1.shape,'\n',perf1.head())
perf2 = perf1.merge(user_num,on = 'date')
perf2['mau'] = perf2['active_num']/perf2['user_num']
perf2['coverage'] = perf2['cover_num']/perf2['user_num']
print('perf2:\n',perf2.shape,'\n',perf2.head())

perf:
         appleid      date  active_num  cover_num
36  953435594.0  20170228        19.0         97
37  953435594.0  20170331        14.0         89
38  953435594.0  20170430        22.0        116
39  953435594.0  20170531        12.0        104
40  953435594.0  20170630        21.0         92
user_num:
 (12, 2) 
        date  user_num
0  20170228    345535
1  20170331    339440
2  20170430    383981
3  20170531    330104
4  20170630    281469
perf1:
 (18625, 4) 
    appleid      date  active_num  cover_num
0  66204.0  20170131         3.0         36
1  66204.0  20170228         0.0         31
2  66204.0  20170331         2.0         29
3  66204.0  20170430         1.0         20
4  66204.0  20170531         2.0         23
perf2:
 (18625, 7) 
        appleid      date  active_num  cover_num  user_num       mau  coverage
0      66204.0  20170131         3.0         36    373183  0.000008  0.000096
1    9223426.0  20170131        10.0         63    373183  0.000027  0.000169
2  281

In [29]:
perf2_vardict = {'datahash': 'varchar(100) primary key',
                 'appleid': 'integer',
                'date':'varchar(20)',
                'active_num':'integer',
                'cover_num':'integer',
                'user_num':'integer',
                'mau':'float',
                'coverage':'float'}
create_table('perf_appleid20190913',perf2_vardict)
insert_data('perf_appleid20190913',perf2)

perf_appleid20190913 created
complete insert


In [30]:
dcgid = dcgpid[['appleid','date','cover_num','g_app_m', 'g_app_v', 'g_apptype_m','g_apptype_v', 'g_gtype_m', 'g_gtype_v', 'g_game_m', 'g_game_v','g_rgame_m', 'g_rgame_v', 'g_rgtype_m', 'g_rgtype_v']]
print('dcgid\n',dcgid.shape,'\n',dcgid.head())
demand_var = ['g_app_m', 'g_app_v', 'g_apptype_m','g_apptype_v', 'g_gtype_m', 'g_gtype_v', 'g_game_m', 'g_game_v','g_rgame_m', 'g_rgame_v', 'g_rgtype_m', 'g_rgtype_v']
for i in demand_var:
    dcgid[i] = dcgid[i]*dcgid['cover_num']
dcgidgroup = dcgid.groupby(['appleid','date'])
dcgid2 = dcgidgroup.agg(sum).reset_index()
for i in demand_var:
    dcgid2[i] = dcgid2[i]/dcgid2['cover_num']
print('dcgid2\n',dcgid2.shape,'\n',dcgid2.head())

dcgid
 (46338, 15) 
         appleid      date  cover_num    g_app_m     g_app_v  g_apptype_m  \
36  953435594.0  20170228         97  17.381443  131.821735    59.742268   
37  953435594.0  20170331         89  16.292135   54.413687    42.078652   
38  953435594.0  20170430        116  16.439655   96.596327    49.068966   
39  953435594.0  20170531        104  16.009615   55.757188    39.846154   
40  953435594.0  20170630         92  13.728261   33.145127    31.554348   

     g_apptype_v  g_gtype_m  g_gtype_v   g_game_m     g_game_v  g_rgame_m  \
36  38975.714132   5.577320   7.246564  13.391753  1686.261598   0.244149   
37   1010.527835   5.337079   5.407814   8.617978    31.329673   0.237631   
38  18387.056072   5.431034   6.212594  11.698276   895.395127   0.248013   
39    759.432412   5.298077   5.298637   9.259615    67.397965   0.248687   
40    270.293717   4.673913   4.485905   7.326087    31.167224   0.240330   

    g_rgame_v  g_rgtype_m  g_rgtype_v  
36   0.015233    0.

/usr/lib/python3/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


dcgid2
 (18625, 15) 
    appleid      date  cover_num    g_app_m     g_app_v  g_apptype_m  \
0  66204.0  20170131         36  19.666667   50.800000    71.500000   
1  66204.0  20170228         31  19.870968   69.516129    74.290323   
2  66204.0  20170331         29  19.103448   78.810345    69.689655   
3  66204.0  20170430         20  20.800000   98.905263    92.550000   
4  66204.0  20170531         23  22.043478  109.225296    84.521739   

   g_apptype_v  g_gtype_m  g_gtype_v   g_game_m    g_game_v  g_rgame_m  \
0  1086.771429   4.861111   4.694444  13.972222  104.256349   0.191001   
1  1827.079570   5.096774   5.623656  15.290323  140.612903   0.203084   
2  1269.793103   5.103448   5.667488  14.862069  111.980296   0.221136   
3  6923.102632   5.800000  10.589474  18.900000  501.042105   0.180027   
4  4908.351779   4.695652   4.312253  13.478261   97.442688   0.184380   

   g_rgame_v  g_rgtype_m  g_rgtype_v  
0   0.008841    0.267497    0.012691  
1   0.009561    0.282075    

In [31]:
demand_game_level_vardict = {'datahash': 'varchar(100) primary key',
                             'appleid': 'integer',
                             'date':'varchar(20)',
                             'cover_num':'integer',
                             'g_app_m':'float',
                             'g_app_v':'float',
                             'g_apptype_m':'float',
                             'g_apptype_v':'float',
                             'g_gtype_m':'float',
                             'g_gtype_v':'float',
                             'g_game_m':'float',
                             'g_game_v':'float',
                             'g_rgame_m':'float',
                             'g_rgame_v':'float',
                             'g_rgtype_m':'float',
                             'g_rgtype_v':'float'}
create_table('game_demand_appleid201909013',demand_game_level_vardict)
insert_data('game_demand_appleid201909013',dcgid2)

game_demand_appleid201909013 created
complete insert


In [32]:
# with open('/home/pi/Documents/buffer/logtemp20190917.txt','r') as f:
#     insertlog(f.read())

## 1.1 生成回归用表1，20190914reg1（基于td分类）

In [48]:
bs1_cmd = 'select * from bs_pkg_span20190912'
diff1_cmd = 'select * from aso_diff20190724'
bs1 = pd.read_sql(bs1_cmd,conn).drop(columns = 'datahash').dropna(subset = ['type_code']).drop(columns = ['type','type_code']).drop_duplicates()
print('bs1: ',bs1.shape)
print('bs1: ',bs1.columns)
diff1 = pd.read_sql(diff1_cmd,conn).drop(columns = 'datahash')
print('diff1: ',diff1.shape)
print('diff1: ',diff1.columns)

bs1:  (1675, 12)
bs1:  Index(['appleid', 'appName', 'price', 'tags', 'rdtimestamp', 'size',
       'payInApp', 'compatibility', 'contentRank', 'samepubappnum', 'age',
       'span'],
      dtype='object')
diff1:  (17046, 8)
diff1:  Index(['appleid', 'type_code', 'date', 'timestamp', 'diff_p_f', 'diff_p_max',
       'diff_e_f', 'diff_e_max'],
      dtype='object')


In [52]:
bs1_diff1 = diff1.merge(bs1, on = 'appleid',how = 'inner')
print(bs1_diff1.shape,'\n',bs1_diff1.head())

(17046, 19) 
       appleid type_code      date   timestamp  diff_p_f  diff_p_max  diff_e_f  \
0  1074628617   T200500  20170131  1485878400  0.934714    0.973451  0.999256   
1  1074628617   T200500  20170228  1488297600  0.955028    0.973418  0.999275   
2  1074628617   T200500  20170331  1490976000  0.962168    0.973429  0.999296   
3  1074628617   T200500  20170430  1493568000  0.964610    0.973497  0.999315   
4  1074628617   T200500  20170531  1496246400  0.963915    0.973585  0.999334   

   diff_e_max                       appName  price          tags  rdtimestamp  \
0    0.776210  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
1    0.776484  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
2    0.776747  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
3    0.777052  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
4    0.777343  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   

     size  p

In [57]:
bs1_diff1_perf2 = bs1_diff1.merge(perf2,on = ['appleid','date'])
print('bs1_diff1_perf2\n',bs1_diff1_perf2.shape,'\n',bs1_diff1_perf2.head())


bs1_diff1_perf2_dc = bs1_diff1_perf2.merge(dc, on = ['type_code','date'], how = 'inner')
print('bs1_diff1_perf2_dc\n',bs1_diff1_perf2_dc.shape,'\n',bs1_diff1_perf2_dc.head())

bs1_diff1_perf2_dc_dcgid2 = bs1_diff1_perf2_dc.merge(dcgid2, on = ['appleid','date','cover_num'],how = 'inner')
print('bs1_diff1_perf2_dc_dcgid2\n',bs1_diff1_perf2_dc_dcgid2.shape,'\n',bs1_diff1_perf2_dc_dcgid2.head())

bs1_diff1_perf2
 (15963, 24) 
       appleid type_code      date   timestamp  diff_p_f  diff_p_max  diff_e_f  \
0  1074628617   T200500  20170131  1485878400  0.934714    0.973451  0.999256   
1  1074628617   T200500  20170228  1488297600  0.955028    0.973418  0.999275   
2  1074628617   T200500  20170331  1490976000  0.962168    0.973429  0.999296   
3  1074628617   T200500  20170430  1493568000  0.964610    0.973497  0.999315   
4  1074628617   T200500  20170531  1496246400  0.963915    0.973585  0.999334   

   diff_e_max                       appName  price          tags  rdtimestamp  \
0    0.776210  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
1    0.776484  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
2    0.776747  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
3    0.777052  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200   
4    0.777343  LEGO® Ninjago: Skybound「已下架」    0.0  游戏、动作游戏、探险游戏   1453651200

In [60]:
bs1_diff1_perf2_dc_dcgid2.columns

Index(['appleid', 'type_code', 'date', 'timestamp', 'diff_p_f', 'diff_p_max',
       'diff_e_f', 'diff_e_max', 'appName', 'price', 'tags', 'rdtimestamp',
       'size', 'payInApp', 'compatibility', 'contentRank', 'samepubappnum',
       'age', 'span', 'active_num', 'cover_num', 'user_num', 'mau', 'coverage',
       'app_m', 'app_v', 'apptype_m', 'apptype_v', 'gtype_m', 'gtype_v',
       'game_m', 'game_v', 'rgame_m', 'rgame_v', 'rgtype_m', 'rgtype_v',
       'type_ratio_m', 'type_ratio_v', 'g_app_m', 'g_app_v', 'g_apptype_m',
       'g_apptype_v', 'g_gtype_m', 'g_gtype_v', 'g_game_m', 'g_game_v',
       'g_rgame_m', 'g_rgame_v', 'g_rgtype_m', 'g_rgtype_v'],
      dtype='object')

In [65]:
# 保存一下
bs1_diff1_perf2_dc_dcgid2_vardict = {'datahash': 'varchar(100) primary key',
                                     'appleid': 'integer','type_code':'varchar(20)','date':'varchar(20)','timestamp':'integer',
                                     'diff_p_f':'float','diff_p_max':'float','diff_e_f':'float','diff_e_max':'float',
                                     'appName': 'varchar(50)','price':'float','tags':'varchar(50)',
                                     'rdtimestamp':'integer','size':'float','payInApp':'integer',
                                     'compatibility':'float','contentRank':'integer','samepubappnum':'integer',
                                     'age':'float','span':'integer',
                                     'active_num':'integer','cover_num':'integer',
                                     'user_num':'integer',
                                     'mau':'float', 'coverage':'float',
                                     'app_m':'float','app_v':'float',
                                     'apptype_m':'float','apptype_v':'float',
                                     'gtype_m':'float','gtype_v':'float',
                                     'game_m':'float', 'game_v':'float',
                                     'rgame_m':'float','rgame_v':'float',
                                     'rgtype_m':'float','rgtype_v':'float',
                                     'type_ratio_m':'float','type_ratio_v':'float',
                                     'g_app_m':'float','g_app_v':'float',
                                     'g_apptype_m':'float','g_apptype_v':'float',
                                     'g_gtype_m':'float','g_gtype_v':'float',
                                     'g_game_m':'float','g_game_v':'float',
                                     'g_rgame_m':'float','g_rgame_v':'float',
                                     'g_rgtype_m':'float','g_rgtype_v':'float'}
create_table('tableForReg_baseOnTdtype20190920',bs1_diff1_perf2_dc_dcgid2_vardict)
insert_data('tableForReg_baseOnTdtype20190920',bs1_diff1_perf2_dc_dcgid2)

tableForReg_baseOnTdtype20190920 created
complete insert


## 1.2 生成回归用表2，20190914reg2（基于自己的分类）

In [70]:
diff2_cmd = 'select * from aso_diff20190913'
diff2 = pd.read_sql(diff2_cmd,conn).drop(columns = 'datahash')
print('diff2:\n',diff2.shape,'\n',diff2.head())

diff2:
 (187144, 6) 
       appleid type_code      date   timestamp  diff_p_f  diff_p_max
0  1117273600   T200900  20170131  1485878400  0.951814    0.870415
1  1117273600   T200600  20170131  1485878400  0.951814    0.870415
2  1117273600   T200900  20170228  1488297600  0.946086    0.866075
3  1117273600   T200600  20170228  1488297600  0.946086    0.866075
4  1117273600   T200900  20170331  1490976000  0.945640    0.854021


In [71]:
bs2 = pd.read_sql(bs1_cmd,conn).drop(columns = ['datahash','type_code','type']).drop_duplicates()
print('bs2:/n',bs2.shape,'\n',bs2.head())

bs2:/n (13088, 12) 
      appleid            appName  price             tags  rdtimestamp   size  \
0  281736535             Enigmo   12.0  游戏、智力游戏、娱乐、街机游戏   1215705600  23.06   
1  284480400     Word Warp「已下架」    6.0     游戏、智力游戏、文字游戏   1215705600  13.65   
2  284752907     Blackjack「已下架」    6.0   游戏、扑克牌游戏、娱乐场游戏   1215705600  51.03   
3  284911691          Dice「已下架」    6.0  游戏、娱乐、桌面游戏、骰子游戏   1215705600    NaN   
4  284943962  Moonlight Mahjong    6.0     游戏、智力游戏、桌面游戏   1215705600  48.20   

   payInApp  compatibility  contentRank  samepubappnum       age  span  
0         1            8.0          4.0             23  9.482192   1.0  
1         0            6.0          4.0             31  9.482192   NaN  
2         1            6.0          4.0             31  9.482192   NaN  
3         0            NaN          NaN              0  9.482192   NaN  
4         0            NaN          4.0              7  9.482192   NaN  


In [72]:
diff2_bs2 = diff2.merge(bs2,on = 'appleid', how = 'left')
print(diff2_bs2.shape,'\n',diff2_bs2.head())

(187144, 17) 
       appleid type_code      date   timestamp  diff_p_f  diff_p_max  \
0  1117273600   T200900  20170131  1485878400  0.951814    0.870415   
1  1117273600   T200600  20170131  1485878400  0.951814    0.870415   
2  1117273600   T200900  20170228  1488297600  0.946086    0.866075   
3  1117273600   T200600  20170228  1488297600  0.946086    0.866075   
4  1117273600   T200900  20170331  1490976000  0.945640    0.854021   

            appName  price            tags  rdtimestamp   size  payInApp  \
0  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
1  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
2  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
3  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
4  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   

   compatibility  contentRank  samepubappnum       age  span  
0            7.0         12.0         

In [74]:
diff2_bs2_perf2 = diff2_bs2.merge(perf2,on = ['appleid','date'], how = 'left')
print(diff2_bs2_perf2.shape,'\n',diff2_bs2_perf2.head())

(187144, 22) 
       appleid type_code      date   timestamp  diff_p_f  diff_p_max  \
0  1117273600   T200900  20170131  1485878400  0.951814    0.870415   
1  1117273600   T200600  20170131  1485878400  0.951814    0.870415   
2  1117273600   T200900  20170228  1488297600  0.946086    0.866075   
3  1117273600   T200600  20170228  1488297600  0.946086    0.866075   
4  1117273600   T200900  20170331  1490976000  0.945640    0.854021   

            appName  price            tags  rdtimestamp   size  payInApp  \
0  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
1  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
2  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
3  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   
4  少女前线-二次元枪娘养成战术手游    0.0  游戏、策略游戏、角色扮演游戏   1467734400  531.0         1   

   compatibility  contentRank  samepubappnum       age  span  active_num  \
0            7.0         

In [75]:
diff2_bs2_perf2_vardict = {'datahash': 'varchar(100) primary key',
                     'appleid': 'integer','type_code':'varchar(20)','date':'varchar(20)','timestamp':'integer',
                     'diff_p_f':'float','diff_p_max':'float',
                     'appName': 'varchar(50)','price':'float','tags':'varchar(50)',
                     'rdtimestamp':'integer','size':'float','payInApp':'integer',
                     'compatibility':'float','contentRank':'integer','samepubappnum':'integer',
                     'age':'float','span':'integer',
                     'active_num':'integer','cover_num':'integer',
                     'user_num':'integer',
                     'mau':'float', 'coverage':'float'}
create_table('tableForRobust_baseOnMytype20190920',diff2_bs2_perf2_vardict)
insert_data('tableForRobust_baseOnMytype20190920',diff2_bs2_perf2)

tableForRobust_baseOnMytype20190920 created
complete insert
